In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.dummy import DummyClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer

In [3]:
train_data = pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv')
test_data = pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv')
sample_data = pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv')
path3='/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv'
sample_data=pd.read_csv(path3)
test_data.shape

(6599, 21)

In [4]:
# Define the columns to use in the analysis
cols = ['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
        'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
        'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
        'GoogleMetric:Page Values', 'SeasonalPurchase',
        'Month_SeasonalPurchase', 'OS', 'SearchEngine', 'Zone',
        'Type of Traffic', 'CustomerType', 'Gender', 'Cookies Setting',
        'Education', 'Marital Status', 'WeekendPurchase','Made_Purchase']

In [5]:

train_data['Gender'] = pd.to_numeric(train_data['Gender'], errors='coerce').fillna(0).astype(int)
train_data['Cookies Setting'] = pd.to_numeric(train_data['Cookies Setting'], errors='coerce').fillna(0).astype(int)
train_data['Education'] = pd.to_numeric(train_data['Education'], errors='coerce').fillna(0).astype(int)
train_data['Marital Status'] = pd.to_numeric(train_data['Marital Status'], errors='coerce').fillna(0).astype(int)
test_data['Gender'] = pd.to_numeric(test_data['Gender'], errors='coerce').fillna(0).astype(int)
test_data['Cookies Setting'] = pd.to_numeric(test_data['Cookies Setting'], errors='coerce').fillna(0).astype(int)
test_data['Education'] = pd.to_numeric(test_data['Education'], errors='coerce').fillna(0).astype(int)
test_data['Marital Status'] = pd.to_numeric(test_data['Marital Status'], errors='coerce').fillna(0).astype(int)
train_data['Month_SeasonalPurchase'] = pd.to_numeric(train_data['Month_SeasonalPurchase'], errors='coerce').fillna(0).astype(int)
test_data['Month_SeasonalPurchase'] = pd.to_numeric(test_data['Month_SeasonalPurchase'], errors='coerce').fillna(0).astype(int)
train_data['CustomerType'] = pd.to_numeric(train_data['CustomerType'], errors='coerce').fillna(0).astype(int)
test_data['CustomerType'] = pd.to_numeric(test_data['CustomerType'], errors='coerce').fillna(0).astype(int)
train_data['Made_Purchase'] = pd.to_numeric(train_data['Made_Purchase'], errors='coerce').fillna(0).astype(int)



In [6]:
imputer = SimpleImputer(strategy='mean')
train_data = imputer.fit_transform(train_data)
test_data = imputer.fit_transform(test_data)


In [7]:
train_data = pd.DataFrame(train_data, columns=cols)
test_data = pd.DataFrame(test_data, columns=cols[:-1])

In [8]:
X_train = train_data[cols[:-1]]
y_train = train_data['Made_Purchase']
X_test = test_data[cols[:-1]]
y_test = sample_data['Made_Purchase']
print(X_train.shape)
print(y_train.shape)

(14731, 21)
(14731,)


In [9]:
scaler = StandardScaler()
print(X_train.shape)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
test_data_scaled = scaler.transform(test_data)

(14731, 21)


In [10]:
selector = SelectKBest(f_classif, k=15)
selector.fit_transform(X_train,y_train)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)
test_data_selected = selector.transform(test_data_scaled)
selected_features = np.array(cols[:-1])[selector.get_support()]
print("Selected features:", selected_features)

Selected features: ['HomePage' 'HomePage_Duration' 'LandingPage' 'LandingPage_Duration'
 'ProductDescriptionPage' 'ProductDescriptionPage_Duration'
 'GoogleMetric:Bounce Rates' 'GoogleMetric:Exit Rates'
 'GoogleMetric:Page Values' 'SeasonalPurchase' 'OS' 'SearchEngine' 'Zone'
 'Type of Traffic' 'WeekendPurchase']


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [10 15 16 17 18 19] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [11]:
logreg = LogisticRegression()
logreg.fit(X_train_selected, y_train)
y_pred = logreg.predict(X_test_selected)
print("Logistic regression accuracy:", accuracy_score(y_test, y_pred))
print("Logistic regression classification report:\n", classification_report(y_test, y_pred))

Logistic regression accuracy: 0.9031671465373542
Logistic regression classification report:
               precision    recall  f1-score   support

         0.0       1.00      0.90      0.95      6599
         1.0       0.00      0.00      0.00         0

    accuracy                           0.90      6599
   macro avg       0.50      0.45      0.47      6599
weighted avg       1.00      0.90      0.95      6599



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
rf = RandomForestClassifier()
rf.fit(X_train_selected, y_train)
y_pred = rf.predict(X_test_selected)
print("Random forest accuracy:", accuracy_score(y_test, y_pred))
print("Random forest classification report:\n", classification_report(y_test, y_pred))

Random forest accuracy: 0.6873768752841339
Random forest classification report:
               precision    recall  f1-score   support

         0.0       1.00      0.69      0.81      6599
         1.0       0.00      0.00      0.00         0

    accuracy                           0.69      6599
   macro avg       0.50      0.34      0.41      6599
weighted avg       1.00      0.69      0.81      6599



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
svm = SVC()
svm.fit(X_train_selected, y_train)
y_pred = svm.predict(X_test_selected)
print("SVM accuracy:", accuracy_score(y_test, y_pred))
print("SVM classification report:\n", classification_report(y_test, y_pred))

SVM accuracy: 0.8811941203212608
SVM classification report:
               precision    recall  f1-score   support

         0.0       1.00      0.88      0.94      6599
         1.0       0.00      0.00      0.00         0

    accuracy                           0.88      6599
   macro avg       0.50      0.44      0.47      6599
weighted avg       1.00      0.88      0.94      6599



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [None, 5, 10, 15]}
rf = RandomForestClassifier()
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_selected, y_train)
print("Best parameters for random forest:", grid_search.best_params_)
rf = grid_search.best_estimator_
y_pred = rf.predict(X_test_selected)
print("Random forest accuracy with best parameters:", accuracy_score(y_test, y_pred))
print("Random forest classification report with best parameters:\n", classification_report(y_test, y_pred))

Best parameters for random forest: {'max_depth': 5, 'n_estimators': 100}
Random forest accuracy with best parameters: 0.8690710713744507
Random forest classification report with best parameters:
               precision    recall  f1-score   support

         0.0       1.00      0.87      0.93      6599
         1.0       0.00      0.00      0.00         0

    accuracy                           0.87      6599
   macro avg       0.50      0.43      0.46      6599
weighted avg       1.00      0.87      0.93      6599



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
sub = pd.DataFrame(
   rf.predict(test_data_selected), 
    columns=['Made_Purchase']
)
sub.index.name = 'id'
sub['Made_Purchase'] = sub['Made_Purchase'].astype('bool') 
sub.to_csv("output.csv", encoding='utf-8')
